In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
df=pd.read_csv('train.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Id
0,2,138,62,35,0,33.6,0.127,47,1,0
1,0,135,68,42,250,42.3,0.365,24,1,3
2,0,173,78,32,265,46.5,1.159,58,0,5
3,4,99,72,17,0,25.6,0.294,28,0,6
4,8,194,80,0,0,26.1,0.551,67,0,7


In [4]:
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Id
0,2,138.0,62,35.0,29.0,33.6,0.127,47,1,0
1,0,135.0,68,42.0,250.0,42.3,0.365,24,1,3
2,0,173.0,78,32.0,265.0,46.5,1.159,58,0,5
3,4,99.0,72,17.0,29.0,25.6,0.294,28,0,6
4,8,194.0,80,23.0,29.0,26.1,0.551,67,0,7


In [6]:
## Create the X and Y data:
X=df.drop(['Outcome','Id'],axis=1)
y=df['Outcome']

In [7]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,2,138.0,62,35.0,29.0,33.6,0.127,47
1,0,135.0,68,42.0,250.0,42.3,0.365,24
2,0,173.0,78,32.0,265.0,46.5,1.159,58
3,4,99.0,72,17.0,29.0,25.6,0.294,28
4,8,194.0,80,23.0,29.0,26.1,0.551,67
...,...,...,...,...,...,...,...,...
1400,6,134.0,70,23.0,130.0,35.4,0.542,29
1401,2,87.0,0,23.0,29.0,28.9,0.773,25
1402,2,75.0,64,24.0,55.0,29.7,0.370,33
1403,6,85.0,78,23.0,29.0,31.2,0.382,42


In [8]:
## Split the date X and Y:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=33)

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10)
rf_classifier.fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [10]:
y.value_counts()

0    918
1    487
Name: Outcome, dtype: int64

In [11]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(accuracy_score(y_test,prediction))

[[174  11]
 [ 16  80]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       185
           1       0.88      0.83      0.86        96

    accuracy                           0.90       281
   macro avg       0.90      0.89      0.89       281
weighted avg       0.90      0.90      0.90       281

0.9039145907473309


In [12]:
## Manual setting Hyperparameters:
model=RandomForestClassifier(n_estimators=300,criterion='entropy',max_features='sqrt',min_samples_leaf=10,random_state=10)
model.fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

[[167  18]
 [ 29  67]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.88       185
           1       0.79      0.70      0.74        96

    accuracy                           0.83       281
   macro avg       0.82      0.80      0.81       281
weighted avg       0.83      0.83      0.83       281

0.8327402135231317


In [13]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest:
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
## Number of features to consider every list:
max_features=['auto','sqrt','log2']
# Maximum number of levels in tree:
max_depth=[int(x) for x in np.linspace(10,100,10)]
# Minimum number of samples required at each node:
min_samples_split=[2,5,10,14]
# Minimum number of samples  required at each leaf node:
min_samples_leaf=[1,2,4,6,8]
# Create the random_grid:
grid={
    'n_estimators': n_estimators,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'criterion':['entropy','gini']
}

print(grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [14]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=grid,n_iter=100,cv=3,verbose=2,n_jobs=-1,random_state=100)
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [15]:
rf_randomcv.best_params_

{'n_estimators': 1400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'criterion': 'entropy'}

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'],
                          rf_randomcv.best_params_['min_samples_split']+1,
                          rf_randomcv.best_params_['min_samples_split']+2, 
                          rf_randomcv.best_params_['min_samples_split']+3,
                          rf_randomcv.best_params_['min_samples_split']+4],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)

{'criterion': ['entropy'], 'max_depth': [20], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [2, 3, 4, 5, 6], 'n_estimators': [1200, 1300, 1400, 1500, 1600]}


In [17]:
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  8.7min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [20],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 3, 4, 5, 6],
                         'n_estimators': [1200, 1300, 1400, 1500, 1600]},
             verbose=2)

In [18]:
grid_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=20, max_features='sqrt',
                       min_samples_split=3, n_estimators=1200)

In [19]:
best_grid=grid_search.best_estimator_

In [20]:
best_grid

RandomForestClassifier(criterion='entropy', max_depth=20, max_features='sqrt',
                       min_samples_split=3, n_estimators=1200)

In [21]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[175  10]
 [ 13  83]]
Accuracy Score 0.9181494661921709
Classification report:               precision    recall  f1-score   support

           0       0.93      0.95      0.94       185
           1       0.89      0.86      0.88        96

    accuracy                           0.92       281
   macro avg       0.91      0.91      0.91       281
weighted avg       0.92      0.92      0.92       281



# Automated Hyperparameter Tuning

# Automated Hyperparameter Tuning can be done by using techniques such as

Bayesian Optimization
Gradient Descent
Evolutionary Algorithms
Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

Objective Function = defines the loss function to minimize.
Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [23]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [33]:
space={
    'criterion':hp.choice('criterion',['entropy','gini']),
    'max_depth':hp.quniform('max_depth',10,1200,10),
    'max_features':hp.choice('max_features',['auto','sqrt','log2',None]),
    'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
    'min_samples_split':hp.uniform('min_samples_split',0,1),
    'n_estimators':hp.choice('n_estimators',[10,50,300,750,1200,1500])
}

In [34]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x1c1845ec340>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x1c1845ec6a0>,
 'max_features': <hyperopt.pyll.base.Apply at 0x1c1845ec8e0>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x1c1845ecdf0>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x1c1845ece50>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x1c1845ec460>}

In [35]:
## Objective:
from sklearn.model_selection import cross_val_score

def objective(space):
    model=RandomForestClassifier(criterion=space['criterion'],
                                max_depth=space['max_depth'],
                                min_samples_leaf=space['min_samples_leaf'],
                                min_samples_split=space['min_samples_split'],
                                n_estimators=space['n_estimators'])
    accuracy=cross_val_score(model,X_train,y_train,cv=5).mean()
    

    ## we aim to maximize accuracy therefore we return it as a negative value:
    return {'loss':-accuracy,'status':STATUS_OK}

In [36]:
from sklearn.model_selection import cross_val_score
trials=Trials()
best=fmin(fn=objective,
         space=space,
         algo=tpe.suggest, ## Suggest is algo
         max_evals=80,
         trials=trials)
best

100%|███████████████████████████████████████████████| 80/80 [04:12<00:00,  3.15s/trial, best loss: -0.7980357142857143]


{'criterion': 1,
 'max_depth': 140.0,
 'max_features': 1,
 'min_samples_leaf': 8.636010802546765e-05,
 'min_samples_split': 0.08269536135805802,
 'n_estimators': 1}

In [37]:
crit={0:'entropy',1:'gini'}
feat={0:'auto',1:'sqrt',2:'log2',3:None}
est={0:10,1:50,2:300,3:750,4:1200,5:1300,6:1500}
print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
sqrt
50


In [38]:
best['min_samples_leaf']

8.636010802546765e-05

In [41]:
trainedforest=RandomForestClassifier(criterion=crit[best['criterion']],
                                max_depth=best['max_depth'],
                                max_features=feat[best['max_features']],
                                min_samples_leaf=best['min_samples_leaf'],
                                min_samples_split=best['min_samples_split'],
                                n_estimators=est[best['n_estimators']]).fit(X_train,y_train)
predictionforest=trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))

[[163  22]
 [ 35  61]]
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       185
           1       0.73      0.64      0.68        96

    accuracy                           0.80       281
   macro avg       0.78      0.76      0.77       281
weighted avg       0.79      0.80      0.79       281

0.797153024911032


# Genetic Algorithms

Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. 
We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). 
We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. 
At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [43]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest:
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
## Number of features to consider every list:
max_features=['auto','sqrt','log2']
# Maximum number of levels in tree:
max_depth=[int(x) for x in np.linspace(10,100,10)]
# Minimum number of samples required at each node:
min_samples_split=[2,5,10,14]
# Minimum number of samples  required at each leaf node:
min_samples_leaf=[1,2,4,6,8]
# Create the random_grid:
grid={
    'n_estimators': n_estimators,
    'max_features':max_features,
    'max_depth':max_depth,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'criterion':['entropy','gini']
}

print(grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [45]:
from tpot import TPOTClassifier

In [47]:
tpot_classifier=TPOTClassifier(generations=5,population_size=24,offspring_size=12,verbosity=2,early_stop=12,
                              config_dict={'sklearn.ensemble.RandomForestClassifier':grid},cv=5,scoring='accuracy')

In [48]:
tpot_classifier.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.9172619047619047

Generation 2 - Current best internal CV score: 0.9172619047619047

Generation 3 - Current best internal CV score: 0.9172619047619047

Generation 4 - Current best internal CV score: 0.9172619047619047

Generation 5 - Current best internal CV score: 0.9172619047619047

Best pipeline: RandomForestClassifier(input_matrix, criterion=entropy, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      20,
                                                                                      30,
                                                                                      40,
                                                                                      50,
                                                                                      60,
                                                                                      70,
                                                                                      80,
                                                                                      90

In [50]:
accuracy=tpot_classifier.score(X_test,y_test)

In [51]:
accuracy

0.9181494661921709